In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
data = pd.read_csv("ag_news.csv", sep=';')
data.head()

,category,content
0,IT,webify helps insurance carriers stretch legac...
1,business,"update 1 thursday #39 s commodities roundup ,..."
2,business,california regulator is suing four insurers a...
3,crime,mexico fta paves way for asian trade agreemen...
4,sport,"t amp t , costa rica through , angus eve scor..."


In [ ]:
print(data.dtypes)

category    object
content     object
dtype: object


In [ ]:
data["category"] = data["category"].astype('category')
data["category"] = data["category"].cat.codes

In [ ]:
data.columns[data.isna().any()].tolist()

[]

Выделяем выходные данные:

In [ ]:
y = data["category"] #DataFrame
data = data['content'] #DataFrame

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=0.3)

На основе тренировочных данных составляем словарь (выписываем все слова, которые встречаются во всех тренировочных текстах). Тестовые данные, как обычно, не трогаем.

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_train):
  for text in data_train:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(data_train), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])  # какой индекс нужно вернуть, если слово не найдено в словаре

In [ ]:
def f():
  for i in [1,6,2,7]:
    yield i  # -> откладывает значения i для возврата, по окончанию работы функции возвращает последовательность всех значений

In [ ]:
list(f())

[1, 6, 2, 7]

unk означает unknown token

vocab - это хитрая штука, не просто набор данных. Пример работы vocab:

In [ ]:
vocab(['feel', 'i', 'say', 'hello', 'hi'])

[2879, 278, 234, 10747, 27047]

In [ ]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, data_text, y):
        self.data = data_text
        self.y = y
    def __len__(self):   # len(trainset)
        return len(self.data)

    def __getitem__(self, idx):
        label = self.y.iloc[idx]  # .iloc потому, что y на самом деле имеет тип не list, а DataFrame
        sentence = vocab(tokenizer(self.data.iloc[idx]))  # .iloc потому, что y на самом деле имеет тип не list, а DataFrame
        return sentence, label

In [ ]:
trainset = TextDataset(data_text=data_train, y=y_train)
testset = TextDataset(data_text=data_test, y=y_test)

In [ ]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(_label)
         processed_text = torch.tensor(_text, dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list, text_list, offsets

In [ ]:
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset, batch_size=8, shuffle=False,
                         collate_fn=collate_batch)

testloader = DataLoader(testset, batch_size=8, shuffle=False,
                        collate_fn=collate_batch)

In [ ]:
import torch
num_class = len(set(y_train))
vocab_size = len(vocab)
print("Кол-во классов: ", num_class)
print("Размер словаря: ", vocab_size)

Кол-во классов:  4
Размер словаря:  81125


Создадим нейронную сеть:

In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self):
        super(TextClassificationModel, self).__init__()
        self.bag = nn.EmbeddingBag(vocab_size, 256, sparse=True)  # слой-матрица размером 81125 x 64, 81125 - такое количество слов в слловаре
        self.lin = nn.Linear(256, 4)  # на вход подаётся 64 числа - выход от self.bag, слой выдаёт 4 числа - вероятность каждой новости, их 4 типа

    def forward(self, text, offsets):
        return self.lin(self.bag(text, offsets))

In [ ]:
net = TextClassificationModel()

In [ ]:
device = 'cuda:0'
net.to(device)

TextClassificationModel(
  (bag): EmbeddingBag(81125, 256, mode='mean')
  (lin): Linear(in_features=256, out_features=4, bias=True)
)

In [ ]:
import torch.optim as optim
# В качестве cost function используем кросс-энтропию
criterion = nn.CrossEntropyLoss()
# В качестве оптимизатора - стохастический градиентный спуск
optimizer = optim.SGD(net.parameters(), lr= 0.5 )

In [ ]:
losses = []
running_corrects = 0
net.train(True)
for epoch in range(5):
    running_loss = 0.0
    running_corrects = 0.0
    for i, data in enumerate(trainloader, 0):
        labels, inputs, offsets = data
        labels = labels.to(device)
        inputs = inputs.to(device)
        offsets = offsets.to(device)

        optimizer.zero_grad()
        outputs = net(inputs, offsets) #
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_corrects += int(torch.sum(preds == labels.data)) / len(labels)
        if i % 10000 == 9999: #
          print('[%d, %5d] loss: %.3f accuracy: %.3f' % (
              epoch + 1, i + 1, running_loss/10000, running_corrects/10000 )) #

          losses += [running_loss/10000]
          running_loss = 0.0
          running_corrects = 0.0

print('Finished Training')

[1, 10000] loss: 0.630 accuracy: 0.767
[2, 10000] loss: 0.423 accuracy: 0.854
[3, 10000] loss: 0.363 accuracy: 0.877
[4, 10000] loss: 0.328 accuracy: 0.890
[5, 10000] loss: 0.303 accuracy: 0.898
Finished Training


In [ ]:
net.train(False)
runninig_correct = 0
num_of_tests = 0
for data in testloader:
    labels, inputs, offsets = data
    labels = labels.to(device)
    inputs = inputs.to(device)
    offsets = offsets.to(device)
    output = net(inputs, offsets)
    _, predicted = torch.max(output, 1)


    runninig_correct += int(torch.sum(predicted == labels)) / len(labels)
    num_of_tests += 1

print(runninig_correct / num_of_tests)

0.8770277777777777
